# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [57]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys1 import geoapify_key

In [58]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,4.16,80,0,1.45,RU,1721940433
1,1,ataq,14.5377,46.8319,29.73,42,98,1.81,YE,1721940433
2,2,blackmans bay,-43.0167,147.3167,7.51,77,84,0.45,AU,1721940433
3,3,castillos,-34.1667,-53.8333,11.91,80,0,1.86,UY,1721940433
4,4,edinburgh of the seven seas,-37.0676,-12.3116,13.72,84,8,17.08,SH,1721940433


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:
%%capture --no-display

city_data_df = pd.read_csv("output_data/cities.csv")

city_data_df['Humidity'] = city_data_df['Humidity'].astype(int)

map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    color='blue',
    alpha=0.75,
    frame_width=800,
    frame_height=600,
    title='City Map with Humidity Levels'
)

map_plot


FileNotFoundError: [Errno 2] No such file or directory: '/opt/anaconda3/lib/python3.11/site-packages/panel/assets/arc_spinner.svg'

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [61]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = pd.read_csv("output_data/cities.csv")

ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
208,208,vallo della lucania,40.2297,15.2658,23.25,71,0,1.96,IT,1721940451
276,276,sejenane,37.0572,9.2381,23.53,77,0,1.77,TN,1721940458
313,313,tolanaro,-25.0319,46.9987,21.04,88,0,0.00,MG,1721940461
321,321,hotan,37.0997,79.9269,26.39,25,0,1.24,CN,1721940462
345,345,rio grande,-32.0350,-52.0986,23.06,71,0,3.48,BR,1721940464


### Step 3: Create a new DataFrame called `hotel_df`.

In [62]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city_data_df = pd.read_csv("output_data/cities.csv")

ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]
ideal_weather_df = ideal_weather_df.dropna()
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()



,City,Country,Lat,Lng,Humidity,Hotel Name
208,vallo della lucania,IT,40.2297,15.2658,71,
276,sejenane,TN,37.0572,9.2381,77,
313,tolanaro,MG,-25.0319,46.9987,88,
321,hotan,CN,37.0997,79.9269,25,
345,rio grande,BR,-32.0350,-52.0986,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [63]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vallo della lucania - nearest hotel: Hotel Mimì
sejenane - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
rio grande - nearest hotel: Hotel Vila Moura Executivo
singaraja - nearest hotel: Hotel Sentral
pula - nearest hotel: Omir
tupaciguara - nearest hotel: Macauba
guelma - nearest hotel: hotel el tadj.
ypsonas - nearest hotel: Club Aphrodite


,City,Country,Lat,Lng,Humidity,Hotel Name
208,vallo della lucania,IT,40.2297,15.2658,71,Hotel Mimì
276,sejenane,TN,37.0572,9.2381,77,No hotel found
313,tolanaro,MG,-25.0319,46.9987,88,Hôtel Mahavokey
321,hotan,CN,37.0997,79.9269,25,Hua Yi International Hotel (accepts foreigners)
345,rio grande,BR,-32.0350,-52.0986,71,Hotel Vila Moura Executivo
353,singaraja,ID,-8.1120,115.0882,80,Hotel Sentral
372,pula,HR,44.8683,13.8481,64,Omir
481,tupaciguara,BR,-18.5922,-48.7050,24,Macauba
497,guelma,DZ,36.4621,7.4261,51,hotel el tadj.
564,ypsonas,CY,34.6884,32.9574,83,Club Aphrodite


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:
%%capture --no-display

data = {
    'city': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'],
    'lat': [40.7128, 34.0522, 41.8781, 29.7604, 33.4484],
    'lon': [-74.0060, -118.2437, -87.6298, -95.3698, -112.0740]
}

df = pd.DataFrame(data)


map_plot = df.hvplot.points(
    'lon',
    'lat',
    geo=True,
    tiles='OSM',
    size=100,  
    color='city',
    alpha=0.75,
    frame_width=800,
    frame_height=600,
    title='City Map'
)

map_plot

FileNotFoundError: [Errno 2] No such file or directory: '/opt/anaconda3/lib/python3.11/site-packages/panel/assets/arc_spinner.svg'

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (city)